In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider

def simulate_ball_on_plate(steps=100, delta_t=0.1, friction=0.98, sx=0.2, sy=0.2, tsx=0.0, tsy=0.0, Kp=4.0, Ki=2.0, Kd=1.0, mu=0.3):
    """
    Simulate a ball on a plate using a PID controller to control the position of the ball.

    Parameters:
    - friction: Friction coefficient of the plate.
    - tsx: Target position in x direction.
    - tsy: Target position in y direction.
    - Kp: Proportional gain for PID controller.
    - Ki: Integral gain for PID controller.
    - Kd: Derivative gain for PID controller.

    The simulation calculates the position and angles of the ball on the plate over a number of steps,
    applying the PID controller to adjust the roll and pitch angles based on the target positions.
    """
    # Initial conditions
    roll = 0.0 # Current roll angle in degrees
    pitch = 0.0 # Current pitch angle in degrees
    # sx = 0.2 # Current Position in meters
    # sy = 0.2 # Current Position in meters
    vx = 0.0 # Current velocity in m/s
    vy = 0.0 # Current velocity in m/s
    g = 9.81 # Gravity g in m/s^2
    # delta_t = 0.1 # Interval in seconds (s)
    max_angle = np.radians(10) # Max angels
    # steps = 200 # Steps

    # Additional varibales
    max_agular_speed = 180.0 # [degree] (degree per second)
    servo_noise_std = 0.1

    # State variables for PID controller
    # Integral and previous error terms for PID control
    integral_x = 0.0
    integral_y = 0.0
    prev_error_x = 0.0
    prev_error_y = 0.0

    # Storage for positions and angles
    time_t = 0
    delta_t_serie = []
    positions_x = []
    positions_y = []
    angles_x = []
    angles_y = []
    delta_x = []
    delta_y = []
    
    # Iterate over the number of steps
    for _ in range(steps):
        time_t += delta_t
        delta_t_serie.append(time_t)
        # Calculate accelerate for x
        roll_theta = roll
        a_max = (5/7) * g * np.sin(roll_theta)
        a_limit = mu * g * np.cos(roll_theta)
        print(f"mu_s >= 2/7 * tan(roll_theta) {mu} >= {2/7 * np.tan(roll_theta):.4f} ({mu >= 2/7 * np.tan(roll_theta)})")
        mu_required = (2/7) * np.tan(roll_theta)
        if mu >= mu_required:
            ax = a_max  # reines Rollen
            print(f"Rollen")
        else:
            ax = a_max - np.sign(a_max) * a_limit  # Rutschen
            print(f"Rutschen")

        # Calculate accelerate for y
        pitch_theta = pitch
        ay = (5/7) * g * (np.sin(pitch_theta) - mu * np.cos(pitch_theta))

        # Calculcate velocity for x
        vx = vx + ax * delta_t

        # Calculcate velocity for y
        vy = vy + ay * delta_t

        # friction
        vx *=friction
        vy *=friction

        # Calculcate position for x
        sx = sx + vx * delta_t + (1/2) * ax * (delta_t * delta_t)

        # Calculcate position for y
        sy = sy + vy * delta_t + (1/2) * ay * (delta_t * delta_t)

        # Degree times delta time
        max_delta_angle = max_agular_speed * delta_t

        # PID for x
        ex = tsx - sx
        integral_x += ex * delta_t
        derivative_x = (ex - prev_error_x) / delta_t
        prev_error_x = ex
        ux = Kp * ex + Ki * integral_x + Kd * derivative_x

        # Set roll
        desire_roll = np.clip(ux, -max_angle, max_angle)
        delta_roll = desire_roll - roll
        roll += np.clip(delta_roll, -max_delta_angle, max_delta_angle)

        positions_x.append(sx)
        angles_x.append(np.degrees(desire_roll))
        delta_x.append(np.clip(delta_roll, -max_delta_angle, max_delta_angle))

        # PID for y
        ey = tsy - sy
        integral_y += ey * delta_t
        derivative_y = (ey - prev_error_y) / delta_t
        prev_error_y = ey
        uy = Kp * ey + Ki * integral_y + Kd * derivative_y

        # # Set pitch
        # Set roll
        desire_pitch = np.clip(uy, -max_angle, max_angle)
        delta_pitch = desire_pitch - pitch
        pitch += np.clip(delta_pitch, -max_delta_angle, max_delta_angle)

        positions_y.append(sy)
        angles_y.append(np.degrees(pitch))
        delta_y.append(np.clip(delta_pitch, -max_delta_angle, max_delta_angle))

    # Plot
    fig, ax = plt.subplots(3, 2, figsize=(10, 8))

    ax[0, 0].plot(delta_t_serie, positions_x)
    ax[0, 0].set(title='Position x',  ylabel='m', xlabel='Zeit (s)')

    ax[1, 0].plot(delta_t_serie, angles_x)
    ax[1, 0].set(title='Rollwinkel',  ylabel='°', xlabel='Zeit (s)')

    ax[2, 0].plot(delta_t_serie, delta_x)
    ax[2, 0].set(title='Delta x',  ylabel='Grad änderung pro Schritt', xlabel='Zeit (s)')
    
    ax[0, 1].plot(delta_t_serie, positions_y)
    ax[0, 1].set(title='Position y',  ylabel='m', xlabel='Zeit (s)')
    
    ax[1, 1].plot(delta_t_serie, angles_y)
    ax[1, 1].set(title='Pitchwinkel', ylabel='°', xlabel='Zeit (s)')

    ax[2, 1].plot(delta_t_serie, delta_y)
    ax[2, 1].set(title='Delta y',  ylabel='Grad änderung pro Schritt', xlabel='Zeit (s)')

    plt.show()

# Interactive widget to control the simulation parameters
interact(simulate_ball_on_plate,
         steps=IntSlider(min=10, max=1000, step=5, value=100, description='steps'),
         delta_t=FloatSlider(min=0.01, max=0.2, step=0.01, value=0.1, description='detla_t (s)'),
         friction=FloatSlider(min=0, max=1, step=0.005, value=0.98, description='Friction'),
         sx=FloatSlider(min=-2, max=2, step=0.1, value=0.0, description='Start X (m)'),
         sy=FloatSlider(min=-2, max=2, step=0.1, value=0.0, description='Start Y (m)'),
         tsx=FloatSlider(min=-2, max=2, step=0.1, value=0.0, description='Target X (m)'),
         tsy=FloatSlider(min=-2, max=2, step=0.1, value=0.0, description='Target Y (m)'),
         Kp=FloatSlider(min=0, max=10, step=0.1, value=1.0, description='Kp'),
         Ki=FloatSlider(min=0, max=1, step=0.01, value=0.0, description='Ki'),
         Kd=FloatSlider(min=0, max=10, step=0.1, value=0.5, description='Kd'),
         mu=FloatSlider(min=0.0, max=0.5, step=0.02, value=0.3, description='mu'))

interactive(children=(IntSlider(value=100, description='steps', max=1000, min=10, step=5), FloatSlider(value=0…

<function __main__.simulate_ball_on_plate(steps=100, delta_t=0.1, friction=0.98, sx=0.2, sy=0.2, tsx=0.0, tsy=0.0, Kp=4.0, Ki=2.0, Kd=1.0, mu=0.3)>